In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Tiny-MLP predictive-cvPCA:
- Train a small, rank-k neural net to map ViT PCs -> brain PCs on TRAIN stimuli.
- Predict on TEST stimuli.
- Do cvPCA-style SVD on (Yhat_test vs Ytrue_test) to get predictive shared spectrum.
- Stimulus K-fold CV, repeat-split for noise, bootstrap CIs, permutation null, noise ceilings.

Author: Maria + Pläku 🐾
"""

import os, pickle, math, time, warnings
import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple, List
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scipy.special import softmax
from skbio.stats.composition import clr

# Optional: torch for the tiny MLP
warnings.filterwarnings("ignore", category=UserWarning)
import torch
import torch.nn as nn
import torch.optim as optim

# -----------------------------
# CONFIG
# -----------------------------
VIT_PATH    = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'

AREAS       = ["VISp","VISl","VISrl","VISal","VISam","VISpm"]

N_IMAGES, N_TRIALS = 118, 50
VAR_VIT, VAR_BRAIN = 0.90, 0.90

K_OUTER     = 5             # stimulus-level CV folds
N_BOOT      = 300           # bootstrap resamples over TEST images (no retraining)
N_PERM      = 50            # permutation null retrains per fold (keep modest)
TOPK_SIGMA  = 10            # how many predictive singulars to plot/save
RANDOM_SEED = 42
OUTDIR      = "results_predictive_cvpca_tinynn"
os.makedirs(OUTDIR, exist_ok=True)

# Tiny MLP hyperparams (very constrained)
BOTTLENECK_K   = 8
NONLIN         = "tanh"     # 'tanh' or 'relu'
DROPOUT        = 0.2
WEIGHT_DECAY   = 1e-2
LR             = 5e-3
EPOCHS         = 1500
PATIENCE       = 120        # early stopping patience
VAL_FRACTION   = 0.2        # inner split from train for early stopping

rng = np.random.default_rng(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# -----------------------------
# Helpers: data prep
# -----------------------------
def vit_pcs(vit_logits: np.ndarray, var=0.90) -> Tuple[np.ndarray, PCA, int]:
    Xv = clr(softmax(np.asarray(vit_logits), axis=1) + 1e-12)
    pfull = PCA(random_state=RANDOM_SEED).fit(Xv)
    n = int(np.searchsorted(np.cumsum(pfull.explained_variance_ratio_), var) + 1)
    p = PCA(n_components=n, random_state=RANDOM_SEED).fit(Xv)
    Zv = p.transform(Xv)
    # z-score PCs for stability
    Zv = (Zv - Zv.mean(0)) / (Zv.std(0) + 1e-8)
    return Zv, p, n

def repeat_split(dat_area: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """(neurons × images × trials × time) -> (even_mean, odd_mean) in (neurons × images)."""
    idx = np.arange(N_TRIALS)
    even, odd = idx[::2], idx[1::2]
    Xe = dat_area[:, :, even, :].mean(axis=(2,3))
    Xo = dat_area[:, :, odd,  :].mean(axis=(2,3))
    return Xe, Xo

def brain_pca_on_train(X_even: np.ndarray, img_tr: np.ndarray, X_odd: np.ndarray,
                       var=0.90) -> Tuple[np.ndarray, np.ndarray, PCA, int]:
    """
    Fit PCA on TRAIN images using EVEN repeats; project EVEN/ODD for ALL images into that basis.
    Return (Ze_all, Zo_all) of shape (N_IMAGES × ncomp), centered per column.
    """
    pfull = PCA(random_state=RANDOM_SEED).fit(X_even[:, img_tr].T)
    n = int(np.searchsorted(np.cumsum(pfull.explained_variance_ratio_), var) + 1)
    p = PCA(n_components=n, random_state=RANDOM_SEED).fit(X_even[:, img_tr].T)
    Ze_all = (X_even.T - p.mean_) @ p.components_.T
    Zo_all = (X_odd.T  - p.mean_) @ p.components_.T
    Ze_all -= Ze_all.mean(0); Zo_all -= Zo_all.mean(0)
    return Ze_all, Zo_all, p, n

# -----------------------------
# Metrics & linear algebra
# -----------------------------
def r2_cols(Y_true: np.ndarray, Y_pred: np.ndarray) -> np.ndarray:
    ss_res = np.sum((Y_true - Y_pred)**2, axis=0)
    ss_tot = np.sum((Y_true - Y_true.mean(axis=0))**2, axis=0) + 1e-12
    return 1.0 - ss_res/ss_tot

def split_half_ceiling(Ze_all: np.ndarray, Zo_all: np.ndarray, img_te: np.ndarray) -> np.ndarray:
    A = Ze_all[img_te] - Ze_all[img_te].mean(0)
    B = Zo_all[img_te] - Zo_all[img_te].mean(0)
    num = np.sum(A*B, axis=0)
    den = np.sqrt(np.sum(A*A, axis=0) * np.sum(B*B, axis=0)) + 1e-12
    rho = np.clip(num/den, -1, 1)
    return (2*rho) / (1 + rho + 1e-12)  # Spearman–Brown

def predictive_cvpca(Yhat_test: np.ndarray, Ytrue_test: np.ndarray) -> np.ndarray:
    """Return singular values of cross-covariance between predicted and true (on test)."""
    C = (Yhat_test.T @ Ytrue_test) / Yhat_test.shape[0]  # (d × d)
    _, S, _ = np.linalg.svd(C, full_matrices=False)
    return S

# -----------------------------
# Tiny rank-k MLP (factorized linear with slight nonlinearity)
# -----------------------------
class RankKMLP(nn.Module):
    def __init__(self, d_in, d_out, k=8, nonlin="tanh", dropout=0.2):
        super().__init__()
        act = {"tanh": nn.Tanh(), "relu": nn.ReLU()}[nonlin]
        self.net = nn.Sequential(
            nn.Linear(d_in, k, bias=False),
            act,
            nn.Dropout(dropout),
            nn.Linear(k, d_out, bias=False),
        )
    def forward(self, x): return self.net(x)

def train_tiny_mlp(Xtr: np.ndarray, Ytr: np.ndarray, *, wd=1e-2, lr=5e-3,
                   epochs=1500, patience=120, k=8, nonlin="tanh", dropout=0.2,
                   val_fraction=0.2) -> RankKMLP:
    """Train with inner train/val split for early stopping. Returns best model (on val)."""
    n = Xtr.shape[0]
    n_val = max(1, int(round(n * val_fraction)))
    idx = np.arange(n)
    rng.shuffle(idx)
    val_idx, tr_idx = idx[:n_val], idx[n_val:]

    Xtr_t = torch.tensor(Xtr[tr_idx], dtype=torch.float32, device=device)
    Ytr_t = torch.tensor(Ytr[tr_idx], dtype=torch.float32, device=device)
    Xva_t = torch.tensor(Xtr[val_idx], dtype=torch.float32, device=device)
    Yva_t = torch.tensor(Ytr[val_idx], dtype=torch.float32, device=device)

    model = RankKMLP(Xtr.shape[1], Ytr.shape[1], k=k, nonlin=nonlin, dropout=dropout).to(device)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.MSELoss()

    best, best_state, wait = float("inf"), None, 0
    for ep in range(epochs):
        model.train()
        opt.zero_grad()
        loss = loss_fn(model(Xtr_t), Ytr_t)
        loss.backward(); opt.step()

        model.eval()
        with torch.no_grad():
            vloss = loss_fn(model(Xva_t), Yva_t).item()
        if vloss < best - 1e-7:
            best = vloss
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

# -----------------------------
# Main
# -----------------------------
def main():
    with open(VIT_PATH, 'rb') as f:
        vit_logits = pickle.load(f)['natural_scenes']
    Zv, vit_pca, d_vit = vit_pcs(vit_logits, VAR_VIT)
    print(f"ViT PCs covering {int(VAR_VIT*100)}% variance: {d_vit}")

    dat   = np.load(NEURAL_PATH, mmap_mode='r')               # (neurons × images*trials*time)
    areas = np.load(AREAS_PATH, allow_pickle=True)

    for AREA in AREAS:
        mask = (areas == AREA)
        dA = dat[mask]
        if dA.size == 0:
            print(f"[WARN] No data for {AREA}; skipping.")
            continue

        print(f"\n=== Area: {AREA} ===")
        n_neu, n_total = dA.shape
        n_time = n_total // (N_IMAGES * N_TRIALS)
        dA = dA.reshape(n_neu, N_IMAGES, N_TRIALS, n_time)

        Xe, Xo = repeat_split(dA)  # even/odd means

        # outer stimulus CV
        stim_idx = np.arange(N_IMAGES)
        kf = KFold(n_splits=K_OUTER, shuffle=True, random_state=RANDOM_SEED)

        fold_R2_mean, fold_R2_norm_mean = [], []
        fold_sigma_pred, fold_p_perm = [], []

        for fold, (i_tr, i_te) in enumerate(kf.split(stim_idx), 1):
            img_tr, img_te = stim_idx[i_tr], stim_idx[i_te]

            # brain PCA in TRAIN
            Ze_all, Zo_all, brain_pca, d_brain = brain_pca_on_train(Xe, img_tr, Xo, VAR_BRAIN)

            # fit tiny MLP on TRAIN stimuli
            Xtr = Zv[img_tr]              # (n_train × d_vit)
            Ytr = Ze_all[img_tr]          # (n_train × d_brain)
            model = train_tiny_mlp(Xtr, Ytr,
                                   wd=WEIGHT_DECAY, lr=LR, epochs=EPOCHS,
                                   patience=PATIENCE, k=BOTTLENECK_K,
                                   nonlin=NONLIN, dropout=DROPOUT, val_fraction=VAL_FRACTION)

            # predict TEST stimuli
            Xte_t = torch.tensor(Zv[img_te], dtype=torch.float32, device=device)
            with torch.no_grad():
                Yhat_test = model(Xte_t).cpu().numpy()        # (n_test × d_brain)
            Ytrue_test = Zo_all[img_te]                       # odd repeats projection

            # metrics
            R2_vec = r2_cols(Ytrue_test, Yhat_test)
            R2_mean = float(np.mean(R2_vec))
            ceiling = split_half_ceiling(Ze_all, Zo_all, img_te)
            R2_norm = np.clip(R2_vec / np.maximum(ceiling, 1e-6), 0, 1)
            R2_norm_mean = float(np.mean(R2_norm))

            # predictive cvPCA spectrum
            S_pred = predictive_cvpca(Yhat_test, Ytrue_test)
            L = min(TOPK_SIGMA, len(S_pred))
            fold_sigma_pred.append(S_pred[:L])

            # bootstrap CIs (no refit)
            S_boot = np.zeros((N_BOOT, L))
            for b in range(N_BOOT):
                idx = rng.integers(0, len(img_te), len(img_te))
                S_b = predictive_cvpca(Yhat_test[idx], Ytrue_test[idx])
                S_boot[b, :min(L, len(S_b))] = S_b[:L]

            # permutation null: shuffle TRAIN pairing, retrain tiny NN, evaluate on same TEST
            S_null = np.zeros((N_PERM, L))
            for b in range(N_PERM):
                perm = rng.permutation(len(img_tr))
                model_n = train_tiny_mlp(Xtr, Ytr[perm],
                                         wd=WEIGHT_DECAY, lr=LR, epochs=EPOCHS//2,  # faster null
                                         patience=max(30, PATIENCE//3),
                                         k=BOTTLENECK_K, nonlin=NONLIN,
                                         dropout=DROPOUT, val_fraction=VAL_FRACTION)
                with torch.no_grad():
                    Yhat_n = model_n(Xte_t).cpu().numpy()
                S_n = predictive_cvpca(Yhat_n, Ytrue_test)
                S_null[b, :min(L, len(S_n))] = S_n[:L]

            # crude p-value for first predictive singular
            p_first = (1 + np.sum(S_null[:, 0] >= S_pred[0])) / (1 + S_null.shape[0])
            fold_p_perm.append(p_first)

            # plot fold spectrum
            x = np.arange(1, L+1)
            lo, hi = np.percentile(S_boot, [2.5, 97.5], axis=0)
            lo_n, hi_n = np.percentile(S_null, [2.5, 97.5], axis=0)
            null_mean = S_null.mean(0)

            plt.figure(figsize=(7,5))
            plt.plot(x, S_pred[:L], 'o-', label='Predictive σ (test)')
            plt.fill_between(x, lo, hi, alpha=0.20, label='Real 95% CI (bootstrap)')
            plt.plot(x, null_mean, 'o--', label='Null mean (perm)')
            plt.fill_between(x, lo_n, hi_n, alpha=0.20, color='gray', label='Null 95% band')
            plt.title(f"{AREA} — fold {fold}  k={BOTTLENECK_K}  R²={R2_mean:.3f}  R²_norm={R2_norm_mean:.3f}")
            plt.xlabel("Predictive component"); plt.ylabel("σ_pred")
            plt.legend(); plt.tight_layout()
            plt.savefig(os.path.join(OUTDIR, f"{AREA}_fold{fold}_predcvpca_tinynn.png"), dpi=150)
            plt.close()

            # save fold bundle
            np.savez(os.path.join(OUTDIR, f"{AREA}_fold{fold}_predcvpca_tinynn.npz"),
                     S_pred=S_pred[:L], S_boot=S_boot, S_null=S_null,
                     R2_mean=R2_mean, R2_norm_mean=R2_norm_mean,
                     img_tr=img_tr, img_te=img_te,
                     k=BOTTLENECK_K, d_vit=d_vit, d_brain=d_brain)

            fold_R2_mean.append(R2_mean)
            fold_R2_norm_mean.append(R2_norm_mean)

            print(f"[{AREA} fold {fold}] R²_mean={R2_mean:.3f}  R²_norm_mean={R2_norm_mean:.3f}  σ1={S_pred[0]:.3f}  p≈{p_first:.3f}")

        # ---- Across-fold summary
        if len(fold_sigma_pred)==0: 
            print(f"[{AREA}] no valid folds.")
            continue
        Lmin = min(len(s) for s in fold_sigma_pred)
        S_stack = np.stack([s[:Lmin] for s in fold_sigma_pred], axis=0)
        S_mean = S_stack.mean(0)

        print(f"[Area {AREA}] mean R²={np.mean(fold_R2_mean):.3f}  mean R²_norm={np.mean(fold_R2_norm_mean):.3f}  σ1_mean={S_mean[0]:.3f}  median p≈{np.median(fold_p_perm):.3f}")

        # save summary
        np.savez(os.path.join(OUTDIR, f"{AREA}_summary_predcvpca_tinynn.npz"),
                 S_mean=S_mean,
                 R2_mean=np.mean(fold_R2_mean),
                 R2_norm_mean=np.mean(fold_R2_norm_mean),
                 p_perm_first=np.median(fold_p_perm))

if __name__ == "__main__":
    main()


ViT PCs covering 90% variance: 44

=== Area: VISp ===
[VISp fold 1] R²_mean=-0.195  R²_norm_mean=0.009  σ1=0.505  p≈0.059
[VISp fold 2] R²_mean=-0.212  R²_norm_mean=0.010  σ1=0.289  p≈0.961
[VISp fold 3] R²_mean=-0.142  R²_norm_mean=0.029  σ1=0.642  p≈0.020
[VISp fold 4] R²_mean=-0.137  R²_norm_mean=0.016  σ1=0.752  p≈0.020
[VISp fold 5] R²_mean=-0.136  R²_norm_mean=0.011  σ1=0.375  p≈0.255
[Area VISp] mean R²=-0.164  mean R²_norm=0.015  σ1_mean=0.513  median p≈0.059

=== Area: VISl ===
[VISl fold 1] R²_mean=-0.062  R²_norm_mean=0.030  σ1=0.517  p≈0.020
[VISl fold 2] R²_mean=-0.073  R²_norm_mean=0.017  σ1=0.372  p≈0.039
[VISl fold 3] R²_mean=-0.089  R²_norm_mean=0.019  σ1=0.378  p≈0.020
[VISl fold 4] R²_mean=-0.102  R²_norm_mean=0.013  σ1=0.352  p≈0.039
[VISl fold 5] R²_mean=-0.128  R²_norm_mean=0.017  σ1=0.352  p≈0.039
[Area VISl] mean R²=-0.091  mean R²_norm=0.019  σ1_mean=0.394  median p≈0.039

=== Area: VISrl ===
[VISrl fold 1] R²_mean=-0.032  R²_norm_mean=0.020  σ1=0.007  p≈0.961
